# Calculating SEDs for the Roman/Rubin 2023 Diffsky Mock

This notebook illustrates how to calculate high-res SEDs of diffsky galaxies, including the SED of the entire galaxy, and also the SED of each galaxy's decomposition into a disk, bulge, and star-forming knots.

First we'll download a very small dataset that stores a downsampling of data from a single healpixel of the `roman_rubin_2023` mock. Then we'll down the template SEDs of the simple stellar populations used to compute the galaxy SEDs.

In [ ]:
! curl https://portal.nersc.gov/project/hacc/aphearin/lsstdesc_diffsky_data/diffsky_z_0_1_cutout_9171.testdata.hdf5 > diffsky.testdata.hdf5

In [ ]:
! curl https://portal.nersc.gov/project/hacc/aphearin/DSPS_data/ssp_data_fsps_v3.2_lgmet_age.h5 > dsps_ssp_data.h5

### Retrieve model parameters

The diffsky model has a number of parameters controlling the galaxy--halo connection. The next cell retrieves the values of these parameters used to generate the `roman_rubin_2023` mock.

In [ ]:
from lsstdesc_diffsky import read_diffskypop_params
all_mock_params = read_diffskypop_params("roman_rubin_2023")

### Load the Diffsky data from the test healpixel

The next cell directly reads the hdf5 file storing the mock data. This test file is formatted in the same way as the healpixels distributed on NERSC: the data is separated by the simulation snapshot, and there is an metadata column storing additional information. For demonstration purposes, we'll just load galaxies directly from the hdf5 file with the `load_diffsky_healpixel` convenience function. DESC users working at NERSC may instead wish to use the [GCR](https://github.com/yymao/generic-catalog-reader).

In [ ]:
from lsstdesc_diffsky.io_utils import load_healpixel

fn = "diffsky.testdata.hdf5"
mock, metadata = load_healpixel(fn)

Each diffsky galaxy has its own individual parameters controlling its assembly history and SED. The `load_diffsky_params` convenience function interprets the columns of the mock that store these parameters, and returns a collection of arrays that are formatted and shaped in the form expected by the SED and photometry convenience functions.

In [ ]:
from lsstdesc_diffsky.io_utils import load_diffsky_params
diffsky_param_data = load_diffsky_params(mock)

### Inspect the SSP data

In [ ]:
from dsps.data_loaders import load_ssp_templates
ssp_data = load_ssp_templates(fn='dsps_ssp_data.h5')

print(ssp_data._fields)

print('\nssp_lgmet.shape = {}'.format(ssp_data.ssp_lgmet.shape))
print('ssp_lg_age_gyr.shape = {}'.format(ssp_data.ssp_lg_age_gyr.shape))
print('ssp_wave.shape = {}'.format(ssp_data.ssp_wave.shape))
print('ssp_flux.shape = {}'.format(ssp_data.ssp_flux.shape))

### Compute the SEDs

In [ ]:
from lsstdesc_diffsky.sed import calc_rest_sed_disk_bulge_knot_galpop
from lsstdesc_diffsky.defaults import OUTER_RIM_COSMO_PARAMS

args = (mock['redshift'],
    diffsky_param_data.mah_params,
    diffsky_param_data.ms_params,
    diffsky_param_data.q_params,
    diffsky_param_data.fbulge_params,
    diffsky_param_data.fknot,
    *ssp_data,
    *all_mock_params,
    OUTER_RIM_COSMO_PARAMS)

disk_bulge_sed_info = calc_rest_sed_disk_bulge_knot_galpop(*args)
print(disk_bulge_sed_info._fields)

### Plot the disk/bulge/knot SEDs of an example galaxy

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(1, 1)
__=ax.loglog()

igal = 9

__=ax.plot(
    ssp_data.ssp_wave, 
    disk_bulge_sed_info.rest_sed_bulge[igal, :], 
    label=r'${\rm bulge}$', color='red')
__=ax.plot(ssp_data.ssp_wave, 
           disk_bulge_sed_info.rest_sed_diffuse_disk[igal, :], 
           label=r'${\rm diffuse disk}$', color='green')
__=ax.plot(ssp_data.ssp_wave, 
           disk_bulge_sed_info.rest_sed_knot[igal, :],
           label=r'${\rm star-forming knots}$', color='purple')

leg = ax.legend()
xlabel = ax.set_xlabel(r'$\lambda\ [{\rm Angstrom}]$')
ylabel = ax.set_ylabel(r'${\rm SED\ [L_{\odot}/Hz]}$')

### Now clean up the temporary files

In [ ]:
! rm dsps_ssp_data.h5

In [ ]:
! rm diffsky.testdata.hdf5